In [ ]:
import pandas as pd

# from nba_api.stats.library.data import teams
# team_id_to_name = {team[0]: team[-3] for team in teams}

In [ ]:
from nba_api.stats.endpoints import *

In [ ]:
from utils import gap_manager

with gap_manager.action_gap():
    game_ids = \
        leaguegamefinder.LeagueGameFinder(league_id_nullable='00').get_data_frames()[0].drop_duplicates(
            subset='GAME_ID',
            keep='first')[
            'GAME_ID']

In [ ]:
from shutil import copytree
from shutil import rmtree
import numpy as np
import json
import pathlib
import youtube_dl
import re

from utils import get_pbp_data, get_video_event_dict, cut_video, add_seconds_to_time

# pandas random relay on numpy random, so this sets the seed for it too
# TODO - remove this eventually
np.random.seed(42)

NUMBER_OF_PLAYS = 10

game_event_action_ids = set()
video_events = list()

p = re.compile('(\s{2}|\' )([\w+ ]*)')

random_game_ids = game_ids.sample(2 * NUMBER_OF_PLAYS, replace=True)
for game_id in random_game_ids:
    df = get_pbp_data(game_id=game_id)

    df['DESCRIPTION'] = df['HOMEDESCRIPTION'].fillna(df['VISITORDESCRIPTION'])
    only_shots = df[df['EVENTMSGTYPE'] <= 2]
    shots_event_data = only_shots[
        ['EVENTNUM', 'EVENTMSGACTIONTYPE', 'PERIOD', 'PCTIMESTRING', 'DESCRIPTION', 'EVENTMSGTYPE',
         'VIDEO_AVAILABLE_FLAG']]

    random_shot_event_data = shots_event_data.sample(1)
    event_id, event_action_id, period, play_clock_time, description, event_msg_type, video_available_flag = \
        random_shot_event_data.iloc[0]
    event_action_id = event_action_id.item()
    if not video_available_flag:
        continue

    game_event_action_ids.add(event_action_id)
    video_event_dict = get_video_event_dict(game_id=game_id, game_event_id=str(event_id))
    video_urls = video_event_dict['resultSets']['Meta']['videoUrls']
    playlist = video_event_dict['resultSets']['playlist']
    video_event = {'desc': playlist[0]['dsc'], 'time': play_clock_time, 'event_action_id': event_action_id,
                   'video': video_urls[0]['lurl'], }
    video_events.append(video_event)

    # if playlist[0]['dsc'] != description:
    # if playlist[0]['dsc'] != description:
    #     raise ValueError(f"{playlist[0]['dsc']} is different that {description}")
    # match = list()
    # match = p.findall(description)
    # event_msg_action = re.sub(' ', '_', match[0][1]).upper()
    print(video_event)

    videos_directory = pathlib.Path('videos')
    video_directory = videos_directory.joinpath(game_id).joinpath(str(event_id))
    video_directory.mkdir(parents=True, exist_ok=True)
    bad_videos_directory = videos_directory.joinpath('Bad Videos')
    bad_videos_directory.mkdir(parents=True, exist_ok=True)
    bad_video_directory = bad_videos_directory.joinpath(game_id).joinpath(str(event_id))

    info_path = video_directory.joinpath('info.json')
    video_path = video_directory.joinpath('video.mp4').as_posix()
    # Save video_event info
    with open(info_path, "w") as outfile:
        json.dump(video_event, outfile)
    # Save video

    ydl_opts = {'outtmpl': video_path}
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_urls[0]['lurl']])

    # Define the duration (in seconds) of the video to be cut after the starting point
    cut_duration = 6
    # If we want a video around `play_clock_time`, we need to start before `play_clock_time`
    # TODO - For inbound plays, we may start the video with the clock too close to play_clock_time, so this wont work
    start_condition = add_seconds_to_time(play_clock_time, seconds_to_add=cut_duration // 1.5)
    output_path = video_directory.joinpath('cut_video.mp4').as_posix()

    print(f"Cutting {video_path} from {start_condition}") # This part can be threaded
    is_recording_successful = cut_video(video_path, start_condition, cut_duration, output_path)
    if not is_recording_successful:
        # We need to move the video to a different folder for farther examination
        copytree(video_directory, bad_video_directory, dirs_exist_ok=True)
        rmtree(video_directory.parent)

    if len(video_events) == NUMBER_OF_PLAYS:
        break

In [ ]:
print(get_video_event_dict.retry.statistics)

In [ ]:
game_event_action_ids

In [ ]:
df = pd.DataFrame(video_events)
df.value_counts('event_action_id')

In [ ]:
# from utils import cut_video, add_seconds_to_time
# import os
# import json
#
# for root, dirs, files in os.walk(r"Videos\Bad Videos", topdown=False):
#     for name in files:
#         if name.endswith('.json'):
#             with open(os.path.join(root, name)) as json_file:
#                 video_event = json.load(json_file)
#                 play_clock_time = video_event['time']
#         elif name == "video.mp4":
#             video_path = os.path.join(root, name)
#
#     # If we want a video around `play_clock_time`, we need to start before `play_clock_time`
#     start_condition = add_seconds_to_time(play_clock_time, seconds_to_add=cut_duration // 1.5)
#     output_path = os.path.join(root, 'cut_video.mp4')
#     is_recording_successful = cut_video(video_path, start_condition, cut_duration, output_path)